In [132]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
#df = pd.read_excel("C:/Users/meharDesktop/Dell/Customer_Segmentation_RFM/Hands_ON/Online_Retail.xlsx")

df = pd.read_excel(r"C:\Users\mehar\Online_Retail.xlsx")
df.head()
df1 = df

ModuleNotFoundError: No module named 'renders'

In [5]:
#Number of unique countries in current dataset
df1.Country.nunique()

38

In [6]:
#List of unique countries in the dataset
df1.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [7]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()

customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [8]:
#From the data, we see that 90%of the customers are from UK. so restricting the data to UK
df1 = df1.loc[df1['Country'] == 'United Kingdom']

In [9]:
#to check the missing values in the dataset. 
df1.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [10]:
#Removing the missing values from the dataset.
df1 = df1[pd.notnull(df1['CustomerID'])]
df1.isnull().sum(axis=0)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [11]:
#Checking the maximum and minimum values in the UnitPrice column
df1.UnitPrice.min()


0.0

In [12]:
df1.UnitPrice.max()

38970.0

In [13]:
print(df1.Quantity.min())
print(df1.Quantity.max())

-80995
80995


In [14]:
#Quantity cannot be negative, Remove these values from dataset
df1 = df1[(df1['Quantity']>0)]
df1.shape
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
InvoiceNo      354345 non-null object
StockCode      354345 non-null object
Description    354345 non-null object
Quantity       354345 non-null int64
InvoiceDate    354345 non-null datetime64[ns]
UnitPrice      354345 non-null float64
CustomerID     354345 non-null float64
Country        354345 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [15]:
#Now we have just 354345 rows and 8 columns

In [131]:
# # DF TO EXCEL
# from pandas import ExcelWriter

# writer = ExcelWriter('Online_Retail_CLEAN.xlsx')
# df1.to_excel(writer,'Sheet5')
# writer.save()


In [16]:
#Unique value for each column
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df1)

InvoiceNo :  16649
StockCode :  3645
Description :  3844
Quantity :  294
InvoiceDate :  15615
UnitPrice :  403
CustomerID :  3921
Country :  1


In [17]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [18]:
#Adding a column for the total price
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

In [19]:
#Finding the first and last order dates in the data
df1['InvoiceDate'].min()


Timestamp('2010-12-01 08:26:00')

In [20]:
df1['InvoiceDate'].max()

Timestamp('2011-12-09 12:49:00')

In [122]:
import datetime
import numpy as np

NOW = dt.datetime(2011,12,10)
#NOW = np.datetime64(NOW)
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

In [81]:
#RFM Customer Segmentation


In [97]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, # Recency
                                        'InvoiceNo': lambda x: len(x),      # Frequency
                                        'TotalPrice': lambda x: x.sum()}) # Monetary Value

# milliseconds = int(round(dt.timestamp() * 1000))
# print(milliseconds

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].values.astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)
                                        
        

In [98]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34


In [125]:
print(rfmTable['recency'].max())
print(rfmTable['frequency'].max())

373
7847


In [99]:
#Checking the details of the first customer
first_customer = df1[df1['CustomerID']== 12346.0]
first_customer

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.6


In [100]:
(NOW - dt.datetime(2011,1,18)).days==326

True

In [127]:
#Split the metrics into segments using quartiles
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75,1])
quantiles
# quantiles = quantiles.to_dict()
# segmented_rfm = rfmTable

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
0.25,17.0,17.0,300.04,1.0,2.0,2.0
0.50,50.0,41.0,651.82,2.0,3.0,3.0
0.75,142.0,99.0,1575.89,3.0,4.0,4.0
1.00,373.0,7847.0,259657.30,4.0,4.0,4.0


In [102]:
quantiles = quantiles.to_dict()
quantiles

{'frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 99.0},
 'monetary_value': {0.25: 300.03999999999996,
  0.5: 651.8200000000002,
  0.75: 1575.8900000000003},
 'recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0}}

In [103]:
segmented_rfm = rfmTable

In [119]:
#Lowest recency and highest frequency and mometary are our best customers
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [121]:
args=('recency',quantiles,)
print(args)

('recency', {'recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0}, 'frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 99.0}, 'monetary_value': {0.25: 300.03999999999996, 0.5: 651.8200000000002, 0.75: 1575.8900000000003}})


In [129]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,4,4,1,441
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4596,33719.73,1,1,1,111
12749.0,3,199,4090.88,1,1,1,111
12820.0,3,59,942.34,1,2,2,122


In [107]:
#Add a new column to combine RFM score
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) \
                            + segmented_rfm.f_quartile.map(str) \
                            + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,4,4,1,441
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4596,33719.73,1,1,1,111
12749.0,3,199,4090.88,1,1,1,111
12820.0,3,59,942.34,1,2,2,122


In [128]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(100)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111


In [ ]:
segmented_rfm.